## Deliverable 2. Create a Customer Travel Destinations Map.

In [66]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [44]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/Znoah/Documents/GitHub/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,49.24,87,3,9.95,clear sky
1,1,Talaya,RU,61.3833,152.7500,56.21,80,99,4.61,overcast clouds
2,2,Aksu,CN,41.1231,80.2644,81.30,36,97,7.83,overcast clouds
3,3,Mataura,NZ,-46.1927,168.8643,28.87,94,17,3.06,few clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,39.31,60,40,11.50,scattered clouds


In [45]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 60


In [46]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Talaya,RU,61.3833,152.7500,56.21,80,99,4.61,overcast clouds
9,9,Mnogovershinnyy,RU,53.9353,139.9242,52.95,99,100,2.26,overcast clouds
11,11,Hermanus,ZA,-34.4187,19.2345,54.81,77,100,13.11,overcast clouds
22,22,Songjianghe,CN,42.1859,127.4790,55.54,97,0,4.52,clear sky
28,28,Port Elizabeth,ZA,-33.9180,25.5701,57.09,98,0,1.01,clear sky
31,31,Yar-Sale,RU,66.8333,70.8333,57.25,74,100,12.35,overcast clouds
40,40,Cape Town,ZA,-33.9258,18.4232,59.88,77,40,10.36,scattered clouds
41,41,Vestmanna,FO,62.1564,-7.1664,52.72,82,20,6.91,few clouds
42,42,Klaksvik,FO,62.2266,-6.5890,52.72,81,100,11.52,overcast clouds
43,43,Vardo,NO,70.3705,31.1107,53.58,87,0,10.36,clear sky


In [48]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [56]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [59]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Talaya,RU,56.21,overcast clouds,61.3833,152.7500,
9,Mnogovershinnyy,RU,52.95,overcast clouds,53.9353,139.9242,
11,Hermanus,ZA,54.81,overcast clouds,-34.4187,19.2345,
22,Songjianghe,CN,55.54,clear sky,42.1859,127.4790,
28,Port Elizabeth,ZA,57.09,clear sky,-33.9180,25.5701,
31,Yar-Sale,RU,57.25,overcast clouds,66.8333,70.8333,
40,Cape Town,ZA,59.88,scattered clouds,-33.9258,18.4232,
41,Vestmanna,FO,52.72,few clouds,62.1564,-7.1664,
42,Klaksvik,FO,52.72,overcast clouds,62.2266,-6.5890,
43,Vardo,NO,53.58,clear sky,70.3705,31.1107,


In [71]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data. // Starter code says google directions, but
    #makes sense to use nearby search instead.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        
        print("Hotel not found... skipping.")
        

In [73]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Mnogovershinnyy,RU,52.95,overcast clouds,53.9353,139.9242,Obshchezhitiye 14
11,Hermanus,ZA,54.81,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
22,Songjianghe,CN,55.54,clear sky,42.1859,127.4790,Sijisheng Fashion Guest House
28,Port Elizabeth,ZA,57.09,clear sky,-33.9180,25.5701,39 On Nile Guest House
31,Yar-Sale,RU,57.25,overcast clouds,66.8333,70.8333,Yalemd
...,...,...,...,...,...,...,...
617,Constitucion,CL,54.90,clear sky,-35.3333,-72.4167,Hotel Boutique Las Azucenas
619,Ostersund,SE,58.60,broken clouds,63.1792,14.6357,Best Western Hotel Gamla Teatern
629,Batemans Bay,AU,53.10,overcast clouds,-35.7167,150.1833,Batemans Bay Marina Resort
651,Matagami,CA,53.71,overcast clouds,49.7502,-77.6328,Motel Le Caribou


In [74]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [80]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# NOTE: Module image of map shows: Hotel Name, City, Country, Current Weather. BUT starter code asks for the above.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [85]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))